In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# !pip install ultralytics
# !git clone https://github.com/ultralytics/yolov5
# %cd yolov5
# !pip install -r requirements.txt

# %cd /content/drive/MyDrive/FinalProjectDeepLearning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.6/599.6 kB 25.0 MB/s eta 0:00:00
fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/drive/.shortcut-targets-by-id/1vDOb7TZctC0hjJ7LI48z26TiJSDQRbb8/FinalProjectDeepLearning/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.7 MB/s eta 0:00:00
/content/drive/.shortcut-targets-by-id/1vDOb7TZctC0hjJ7LI48z26TiJSDQRbb8/FinalProjectDeepLearning


In [3]:
import os

# Change directory to "FinalProjectDeepLearning"
os.chdir('/content/drive/MyDrive/FinalProjectDeepLearning')

import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import itertools
import pandas as pd

from ultralytics import YOLO

import xml.etree.ElementTree as ET

import shutil

import random

from google.colab.patches import cv2_imshow

In [4]:
data_path = '/content/drive/MyDrive/FinalProjectDeepLearning/Dataset'
full_ijcnn_path = '/content/drive/MyDrive/FinalProjectDeepLearning/FullIJCNN2013'
test_videos_path = '/content/drive/MyDrive/FinalProjectDeepLearning/TestVideos'

In [8]:
# LABELS

trafficlight = 0
speedlimit = 1
crosswalk = 2
stop = 3

In [ ]:
# FIRST DATASET DATA CONVERSION

def convert_box(size, box):
    dw, dh = 1. / size[0], 1. / size[1]
    x, y, w, h = (box[0] + box[1]) / 2.0 - 1, (box[2] + box[3]) / 2.0 - 1, box[1] - box[0], box[3] - box[2]
    return x * dw, y * dh, w * dw, h * dh

def convert_voc_to_yolo():
    for anno in os.listdir(data_path+'/labels'):
        if anno.split('.')[1] == 'xml':
            file_name = anno.split('.')[0]
            out_file = open(f'{data_path}/labels/{file_name}.txt', 'w')

            tree = ET.parse(os.path.join('Dataset','labels', anno))
            root = tree.getroot()
            size = root.find('size')
            w = int(size.find('width').text)
            h = int(size.find('height').text)

            names = ['trafficlight', 'speedlimit', 'crosswalk', 'stop']

            for obj in root.iter('object'):
                cls = obj.find('name').text
                if cls in names and int(obj.find('difficult').text) != 1:
                    xmlbox = obj.find('bndbox')
                    bb = convert_box((w, h), [float(xmlbox.find(x).text) for x in ('xmin', 'xmax', 'ymin', 'ymax')])
                    cls_id = names.index(cls)  # class id
                    out_file.write(" ".join([str(a) for a in (cls_id, *bb)]) + '\n')

#convert_voc_to_yolo()

In [ ]:
# FULLIJCNN DATA CONVERSION

def convert_ppm_to_png(ppm_path, png_path):
    image = cv2.imread(ppm_path + '.ppm', cv2.IMREAD_UNCHANGED)
    cv2.imwrite(png_path + '.png', image)

def get_image_dimensions(image):
    height, width = image.shape[:2]
    return width, height

def create_text_file(file_path, line_content):
    with open(file_path+'.txt', 'w') as file:
        file.write(line_content)

def convert_IJCNN_to_dataset():
  full_ijcnn_labels_to_speedlimit = [0,1,2,3,4,5,7,8]
  full_ijcnn_labels_to_stop = 14

  with open(full_ijcnn_path + '/gt.txt', 'r') as file_:
      lines_full_ijcnn  = file_.readlines()

  for line in lines_full_ijcnn:
    splited_line = line.split(';')

    file_name = splited_line[0][:-4]
    left_col = int(splited_line[1])
    top_row = int(splited_line[2])
    right_col = int(splited_line[3])
    bottom_row = int(splited_line[4])
    label = int(splited_line[5])

    if label == full_ijcnn_labels_to_stop:
      label = stop
    elif label in full_ijcnn_labels_to_speedlimit:
      label = speedlimit
    else:
      label = -1

    if label >= 0:
      image = cv2.imread(full_ijcnn_path + '/' + file_name + '.ppm')
      width, height = get_image_dimensions(image)

      bb = convert_box((width, height), (left_col, right_col, top_row, bottom_row))

      with open(f'{data_path}/labels/{file_name}.txt', 'a') as out_file:
        out_file.write(" ".join([str(a) for a in (label, *bb)]) + '\n')

      convert_ppm_to_png(full_ijcnn_path + '/' + file_name, data_path + '/images/' + file_name)

#convert_IJCNN_to_dataset()

In [ ]:
# ELIMINAR IMAGENES EN PARTICION DATASET

def clear_folder(folder_path):
  for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
      os.remove(file_path)
    elif os.path.isdir(file_path):
      clear_folder(file_path)
      os.rmdir(file_path)

#clear_folder(data_path + '/images') # ELIMINAR IMAGENES EN ORIGINAL
#clear_folder(data_path + '/labels') # ELIMINAR IMAGENES EN ORIGINAL

#clear_folder(data_path + '/train/images') # ELIMINAR IMAGENES EN TRAIN
#clear_folder(data_path + '/train/labels') # ELIMINAR IMAGENES EN TRAIN

#clear_folder(data_path + '/validation/images') # ELIMINAR IMAGENES EN VALIDATION
#clear_folder(data_path + '/validation/labels') # ELIMINAR IMAGENES EN VALIDATION

#clear_folder(data_path + '/test/images') # ELIMINAR IMAGENES EN TEST
#clear_folder(data_path + '/test/labels') # ELIMINAR IMAGENES EN TEST

In [ ]:
def contains_string(target_string, string_list):
  for string in string_list:
    if string in target_string:
      return True
  return False

def data_division():
  imagesForDA = os.listdir(data_path + 'images')
  labelsForDA = os.listdir(data_path + 'labels')
  list_of_validation_samples = []
  list_of_test_samples = []

  for fileNameImage in imagesForDA:
    if contains_string(fileNameImage, list_of_validation_samples):
      shutil.move(data_path + 'images/'+fileNameImage, data_path + 'validation/images/'+fileNameImage)
    elif contains_string(fileNameImage, list_of_validation_samples):
      shutil.move(data_path + 'images/'+fileNameImage, data_path + 'test/images/'+fileNameImage)
    else:
      shutil.move(data_path + 'images/'+fileNameImage, data_path + 'train/images/'+fileNameImage)

  for labelName in labelsForDA:
    if labelName.endswith(".txt"):
      if contains_string(labelName, list_of_validation_samples):
        shutil.move(data_path + 'labels/'+labelName, data_path + 'validation/labels/'+labelName)
      elif contains_string(labelName, list_of_validation_samples):
        shutil.move(data_path + 'labels/'+labelName, data_path + 'validation/labels/'+labelName)
      else:
        shutil.move(data_path + 'labels/'+labelName, data_path + 'test/labels/'+labelName)

# DEVOLVER IMAGENES A ORIGINAL

def return_to_original():
  train_images = os.listdir(data_path + '/train/images')
  train_labels = os.listdir(data_path + '/train/labels')

  validation_images = os.listdir(data_path + '/validation/images')
  validation_labels = os.listdir(data_path + '/validation/labels')

  test_images = os.listdir(data_path + '/test/images')
  test_labels = os.listdir(data_path + '/test/labels')

  for fileNameImage in train_images:
    shutil.move(data_path + '/train/images/'+fileNameImage, data_path + '/images/'+fileNameImage)

  for fileNameImage in validation_images:
    shutil.move(data_path + '/validation/images/'+fileNameImage, data_path + '/images/'+fileNameImage)

  for fileNameImage in test_images:
    shutil.move(data_path + '/test/images/'+fileNameImage, data_path + '/images/'+fileNameImage)

  for fileLabel in train_labels:
    shutil.move(data_path + '/train/labels/'+fileLabel, data_path + '/labels/'+fileLabel)

  for fileLabel in validation_labels:
    shutil.move(data_path + '/validation/labels/'+fileLabel, data_path + '/labels/'+fileLabel)

  for fileLabel in test_labels:
    shutil.move(data_path + '/test/labels/'+fileLabel, data_path + '/labels/'+fileLabel)

#OTHER WAY TO DO DATA DIVISION

def divide_list_random(list_data, percentage):
    random.shuffle(list_data)
    length = len(list_data)
    split_index = int(length * percentage / 100)
    list_part1 = list_data[:split_index]
    list_part2 = list_data[split_index:]
    return list_part1, list_part2

def data_division_with_percentage():
  imagesForDA = os.listdir(data_path + '/images')
  labelsForDA = os.listdir(data_path + '/labels')

  labels = []

  for label in imagesForDA:
    labels.append(label[:-4])

  labels_train, labels_validation = divide_list_random(labels, 90)
  labels_validation, labels_test = divide_list_random(labels_validation, 50)

  for name in labels_train:
    shutil.move(data_path + '/images/'+name+'.png', data_path + '/train/images/'+name+'.png')
    shutil.move(data_path + '/labels/'+name+'.txt', data_path + '/train/labels/'+name+'.txt')

  for name in labels_validation:
    shutil.move(data_path + '/images/'+name+'.png', data_path + '/validation/images/'+name+'.png')
    shutil.move(data_path + '/labels/'+name+'.txt', data_path + '/validation/labels/'+name+'.txt')

  for name in labels_test:
    shutil.move(data_path + '/images/'+name+'.png', data_path + '/test/images/'+name+'.png')
    shutil.move(data_path + '/labels/'+name+'.txt', data_path + '/test/labels/'+name+'.txt')

#return_to_original()
#data_division_with_percentage()

In [9]:
%cd /content/drive/MyDrive/FinalProjectDeepLearning

/content/drive/.shortcut-targets-by-id/1vDOb7TZctC0hjJ7LI48z26TiJSDQRbb8/FinalProjectDeepLearning


In [10]:
import os
import glob

# Function to delete files with exceptions
def delete_exception_files(exception_path, directory):
    # Load exceptions from the file
    with open(exception_path, 'r') as file:
        exceptions = file.read().splitlines()

    # Go through each exception
    for exception in exceptions:
        exception = exception.strip(',') # Remove trailing comma if it exists
        # Look for any file that includes the exception in its name, regardless of its extension
        for filepath in glob.glob(directory + f'/**/*{exception}.*', recursive=True):
            print(f'Deleting: {filepath}')
            os.remove(filepath)

# Use the function
delete_exception_files('EXCEPCIONES.txt', './Dataset')


Deleting: ./Dataset/train/images/00054.png
Deleting: ./Dataset/train/labels/00054.txt
Deleting: ./Dataset/train/images/00115.png
Deleting: ./Dataset/train/labels/00115.txt
Deleting: ./Dataset/train/images/00130.png
Deleting: ./Dataset/train/labels/00130.txt
Deleting: ./Dataset/train/images/00131.png
Deleting: ./Dataset/train/labels/00131.txt
Deleting: ./Dataset/train/images/00141.png
Deleting: ./Dataset/train/labels/00141.txt
Deleting: ./Dataset/train/images/00165.png
Deleting: ./Dataset/train/labels/00165.txt
Deleting: ./Dataset/train/images/00172.png
Deleting: ./Dataset/train/labels/00172.txt
Deleting: ./Dataset/train/images/00177.png
Deleting: ./Dataset/train/labels/00177.txt
Deleting: ./Dataset/train/images/00202.png
Deleting: ./Dataset/train/labels/00202.txt
Deleting: ./Dataset/train/images/00214.png
Deleting: ./Dataset/train/labels/00214.txt
Deleting: ./Dataset/train/images/00222.png
Deleting: ./Dataset/train/labels/00222.txt
Deleting: ./Dataset/train/images/00226.png
Deleting: .

In [5]:
# Freeze
def freeze_layer(trainer):
    model = trainer.model
    num_freeze = 10
    print(f"Freezing {num_freeze} layers")
    freeze = [f'model.{x}.' for x in range(num_freeze)]  # layers to freeze
    for k, v in model.named_parameters():
        v.requires_grad = True  # train all layers
        if any(x in k for x in freeze):
            print(f'freezing {k}')
            v.requires_grad = False
    print(f"{num_freeze} layers are freezed.")

# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)
# model = YOLO("yolov5s.yaml")  # build a new model from scratch
# model = YOLO("yolov5su.pt")  # load a pretrained model (recommended for training)

#model.add_callback("on_train_start", freeze_layer)


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [6]:
model.train(data="TrafficVOC.yaml", epochs=50)  # train the model
metrics = model.val()  # evaluate model performance on the validation set

Ultralytics YOLOv8.0.117 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=TrafficVOC.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, si

In [15]:
model.export()

Ultralytics YOLOv8.0.117 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU

PyTorch: starting from runs/detect/train6/weights/best.pt with input shape (16, 3, 640, 640) BCHW and output shape(s) (16, 8, 8400) (17.6 MB)

TorchScript: starting export with torch 2.0.1+cu118...
TorchScript: export success ✅ 42.9s, saved as runs/detect/train6/weights/best.torchscript (35.2 MB)

Export complete (66.2s)
Results saved to /content/drive/.shortcut-targets-by-id/1vDOb7TZctC0hjJ7LI48z26TiJSDQRbb8/FinalProjectDeepLearning/runs/detect/train6/weights
Predict:         yolo predict task=detect model=runs/detect/train6/weights/best.torchscript imgsz=640 
Validate:        yolo val task=detect model=runs/detect/train6/weights/best.torchscript imgsz=640 data=TrafficVOC.yaml 
Visualize:       https://netron.app


'runs/detect/train6/weights/best.torchscript'

In [7]:
model = YOLO("runs/detect/train8/weights/best.pt")

In [8]:
model.predict(test_videos_path+"/TestVideo1.mp4", save=True)



    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/1371) /content/drive/MyDrive/FinalProjectDeepLearning/TestVideos/TestVideo1.mp4: 384x640 1 trafficlight, 59.3ms
video 1/1 (2/1371) /content/drive/MyDrive/FinalProjectDeepLearning/TestVideos/TestVideo1.mp4: 384x640 1 trafficlight, 7.7ms
video 1/1 (3/1371) /content/drive/MyDrive/FinalProjectDeepLearning/TestVideos/TestVideo1.mp4: 384x640 1 trafficlight, 6.5ms
video 1/1 (4/1371) /content/drive/MyDrive/FinalProjectDeepLearning/TestVideos/TestVide

[ultralytics.yolo.engine.results.Results object with attributes:
 
 boxes: ultralytics.yolo.engine.results.Boxes object
 keypoints: None
 keys: ['boxes']
 masks: None
 names: {0: 'trafficlight', 1: 'speedlimit', 2: 'crosswalk', 3: 'stop'}
 orig_img: array([[[217, 197, 199],
         [217, 197, 199],
         [217, 197, 199],
         ...,
         [  6,   2,   5],
         [  6,   2,   5],
         [  6,   2,   5]],
 
        [[217, 197, 199],
         [217, 197, 199],
         [217, 197, 199],
         ...,
         [  6,   2,   5],
         [  6,   2,   5],
         [  6,   2,   5]],
 
        [[217, 197, 199],
         [217, 197, 199],
         [217, 197, 199],
         ...,
         [  6,   2,   5],
         [  6,   2,   5],
         [  6,   2,   5]],
 
        ...,
 
        [[ 32,  29,  30],
         [ 32,  29,  30],
         [ 32,  29,  30],
         ...,
         [ 19,  15,  18],
         [ 19,  15,  18],
         [ 19,  15,  18]],
 
        [[ 32,  29,  30],
         [ 32,  29

In [ ]:
#!python yolov5/train.py --img 320 --batch 16 --epochs 25 --data TrafficVOC.yaml --weights yolov5s.pt --workers 2 --freeze 10

In [ ]:
#model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp4/weights/best.pt', force_reload=True)

In [ ]:
#path = model.export()  # export the model to pt format

In [32]:
# PASAR UN VIDEO -> SUBIR EL VIDEO, DIVIDIRLO EN FOTOGRAMAS, Y PASARLE CADA FOTOGRAMA POR EL VIDEO
"""
def predict_video(video_path, output_path):
  cap = cv2.VideoCapture(video_path)

  #OUTPUT VIDEO
  frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = cap.get(cv2.CAP_PROP_FPS)
  codec = cv2.VideoWriter_fourcc(*"mp4v")
  output_video = cv2.VideoWriter(output_path, codec, fps, (frame_width, frame_height))

  while cap.isOpened():
      ret, frame = cap.read()

      if not ret:
          break

      # RESIZE IMAGE
      height, width = frame.shape[:2]
      frame = cv2.resize(frame, (2 * width, 2 * height))

      result = model(frame)

      output_video.write(np.squeeze(cv2.resize(np.squeeze(result.render()),(width, height))))

  cap.release()
  output_video.release()
  cv2.destroyAllWindows()

model.conf = 0.25
predict_video(test_videos_path + '/TestVideo1.mp4', test_videos_path + '/TestVideo1_Yolov5_ultima_oportunidad.mp4')
"""


0: 384x640 1 stop, 15.0ms
Speed: 2.7ms preprocess, 15.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)


error: ignored

In [ ]:
model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("/content/drive/MyDrive/FinalProjectDeepLearning/Results/TRAINS/TRAIN_YOLOv8_Freeze_1096imgs/weights/best.pt")  # load a pretrained model (recommended for training)




                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

[ultralytics.yolo.engine.results.Results object with attributes:
 
 boxes: ultralytics.yolo.engine.results.Boxes object
 keypoints: None
 keys: ['boxes']
 masks: None
 names: {0: 'trafficlight', 1: 'speedlimit', 2: 'crosswalk', 3: 'stop'}
 orig_img: array([[[217, 197, 199],
         [217, 197, 199],
         [217, 197, 199],
         ...,
         [  6,   2,   5],
         [  6,   2,   5],
         [  6,   2,   5]],
 
        [[217, 197, 199],
         [217, 197, 199],
         [217, 197, 199],
         ...,
         [  6,   2,   5],
         [  6,   2,   5],
         [  6,   2,   5]],
 
        [[217, 197, 199],
         [217, 197, 199],
         [217, 197, 199],
         ...,
         [  6,   2,   5],
         [  6,   2,   5],
         [  6,   2,   5]],
 
        ...,
 
        [[ 32,  29,  30],
         [ 32,  29,  30],
         [ 32,  29,  30],
         ...,
         [ 19,  15,  18],
         [ 19,  15,  18],
         [ 19,  15,  18]],
 
        [[ 32,  29,  30],
         [ 32,  29